In [ ]:
!pip install numpy==1.25.2
!pip install torch==2.7.0
!pip install torchvision==0.22.0
!pip install gym_super_mario_bros==7.4.0
!pip install gym==0.26.2
!pip install nes_py==8.2.1
!pip install gymnasium==0.29.1
!pip install opencv-python==4.11.0.86

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import torch,gym,random
from dataclasses import dataclass
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation,FrameStack,ResizeObservation
from gym.vector import SyncVectorEnv
import numpy as np

@dataclass(frozen=False)
class hypers:
    seed : int = 42
    lambda_ : float = 1.0
    gamma : float = 0.99
    epsilon : float = 0.2
    lr : float = 1e-4
    critic_coeff : float = 1e-1
    policy_ceoff : int = 1.0
    entropy_coeff : float = 1e-3
    skip_frame : int = 4
    num_stack : int  = 4
    obs_shape : tuple[int,int] = (100,100) # observation shape
    num_env : int = 90 
    num_game : int = 500  
    batchsize : int =  512 
    minibatch : int = 16
    optim_steps : int = 10
    # 90*500*512 ~ 23 M steps, training target = 20 M 

configs = hypers()

torch.manual_seed(configs.seed)
random.seed(configs.seed)
np.random.seed(configs.seed)
xm.set_rng_state(configs.seed)

In [ ]:
class CustomEnv(gym.Wrapper): 
        def __init__(self,env,skip):
            super().__init__(env)
            self.skip = skip
            self.score = 0
            
        def step(self, action):
            total_reward = 0  
            for _ in range(self.skip):
                obs,reward,done,truncared,info = self.env.step(action)
                total_reward += reward + ((info.get("score") - self.score ) / 10.) 
                self.score = info.get("score")
                if done:
                    if info.get("flag_get"):
                        total_reward += 50
                    else :
                        total_reward -= 50
                    self.reset()
                    return obs,(total_reward/10.),done,truncared,info
            return obs,(total_reward/10.),done,truncared,info

        def reset(self, **kwargs):
            self.score = 0
            obs, info = self.env.reset()
            return obs,info

def make_env(): 
    x = gym_super_mario_bros.make("SuperMarioBros-v0",apply_api_compatibility=True) 
    x = ResizeObservation(x,configs.obs_shape)  
    x = CustomEnv(x,configs.skip_frame) 
    x = JoypadSpace(x, SIMPLE_MOVEMENT)  
    x = GrayScaleObservation(x,True)
    x = FrameStack(x,configs.num_stack)
    x = SyncVectorEnv(env_fns=[lambda : x] * configs.num_env)
    return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4,32,1,1,0)
        self.conv2 = nn.Conv2d(32,32,3,2,2)
        self.conv3 = nn.Conv2d(32,32,3,2,2)
        self.conv4 = nn.Conv2d(32,32,3,2,2)
        self.output = nn.Linear(169,80)

        self.policy_head = nn.Linear(80,7)
        self.value_head = nn.Linear(80,1)
        self.optim = torch.optim.Adam(self.parameters(),lr=1)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = torch.flatten(x,start_dim=1) # -> torch.Size([32, 169])
        x = F.relu(self.output(x))
        policy_output = self.policy_head(x)
        value_output = self.value_head(x)
        return F.softmax(policy_output,-1),value_output 

In [ ]:
import gc,sys
from torch.distributions import Categorical
from collections import deque

class Memory:
    def __init__(self,env : SyncVectorEnv):
        self.network = network()
        self.env = env
        self.gamma = configs.gamma
        self._lambda_ = configs.lambda_
        self.data = []
        self.pointer = 0
        self.episode_reward = np.zeros(self.env.num_envs, dtype=np.float32)
        self.finished_reward = deque(maxlen=5)
        self.log_total_steps = deque(maxlen=1)
        self.total_steps = torch.zeros(configs.num_env)
    
    def rollout(self,batchsize,device):
        self.clear()
        self._observation,_ = self.env.reset()
        with torch.no_grad():
            for n in range(batchsize):
                self._observation = torch.from_numpy(np.array(self._observation).copy()).squeeze(-1).to(device,torch.float32) / 255.
                policy_output , value = self.network.forward(self._observation)
                distribution = Categorical(policy_output)
                action = distribution.sample()
                prob = distribution.log_prob(action)
                state,reward,done,_,_ = self.env.step(action.tolist())
                
                for i in range(self.env.num_envs): # reward per episode in a deque
                    self.episode_reward[i] += reward[i] 
                    self.total_steps[i] += 1
                    if done[i]:
                        self.finished_reward.append(self.episode_reward[i])
                        self.log_total_steps.append(self.total_steps[i])
                        self.episode_reward[i] = 0
                        self.total_steps[i] = 0
        
                self.data.append(
                    [
                        self._observation,
                        torch.tensor(reward,dtype=torch.float32),
                        value,
                        prob,
                        action,
                        done
                    ]
                )
                self._observation = state
 
        _,rewards,values,_,_,_ = zip(*self.data) # advantages 
        _rewards = torch.stack(rewards).to(device) 
        _values = torch.stack(values).squeeze(-1) 

        zeros = torch.zeros(1,_values.shape[-1],device=device,dtype=torch.float32)
        _values = torch.cat([_values,zeros])
        assert _rewards.dtype == _values.dtype == torch.float32 
        advantages = torch.zeros_like(_rewards,device=device,dtype=torch.float32)
        gae = 0.0
        for n in reversed(range(len(_rewards))):
            td = _rewards[n] + self.gamma * _values[n+1] - _values[n] 
            gae = td + (self._lambda_ * self.gamma * gae) 
            advantages[n] = gae

        for data,item in zip(self.data,advantages): # append advantages to data
            data.append(item)
            
        random.shuffle(self.data) 

    def sample(self,number):
        output = self.data[self.pointer:self.pointer+number]
        self.pointer+=number
        states,rewards,values,logProb,actions,done,advantages = zip(*output)
        return states,actions,rewards,values,logProb,advantages,done

    def clear(self):
        self.data = []
        self.pointer = 0
        gc.collect()
        torch.cuda.empty_cache()
    
    def traj_reward(self):
        return self.finished_reward,self.log_total_steps

In [ ]:
torch.autograd.set_detect_anomaly(True)
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

class agent:
    def __init__(self,device):
        self.env = make_env()
        self.memory = Memory(self.env)
        self.device = device
        self.network = network().to(self.device)
        self.writter = SummaryWriter("./")
        

    def save(self,k):
        checkpoint = {
            "model_state" : self.network.state_dict(),
            "optim_state" : self.network.optim.state_dict(),
        }
        torch.save(checkpoint,f"./mario{k}")

    def train(self,num_game,batchsize,minibatch,optim_step):
        for traj in tqdm(range(num_game),total = num_game):
            self.memory.rollout(batchsize,self.device)

            for _ in range(batchsize//minibatch):
                states,actions,rewards,values,old_log_prob,advantages,done= self.memory.sample(minibatch)    
                 
                for _ in range(optim_step): 
                    _advantages = torch.stack(advantages).view(-1)
                    _values = torch.stack(values).squeeze().view(-1)
                    _old_log_prob = torch.stack(old_log_prob) 
                    
                    vtarget = _advantages + _values
                    loss_critic = F.smooth_l1_loss(_values,vtarget) * 1e-1
                    explained_variance = 1.0 - (torch.var(vtarget - _values) / torch.var(vtarget + 1e-10))
                
                    stacked_states = torch.stack(states) # - > [minibatch, num_env, channel, image_shape]
                    flat_states = torch.flatten(stacked_states,0,1).to(self.device) # - > [(minibatch * num_env), channel, image_shape]
                    p_output,_ = self.network.forward(flat_states)
                    policy_output = p_output.view(minibatch,configs.num_env,7) # 7 is the actionspace for a single env
                    dist = Categorical(policy_output)  
                    stacked_actions = torch.stack(actions)  
                    new_log_prob = dist.log_prob(stacked_actions)

                    ratio = torch.exp(new_log_prob - _old_log_prob).view(-1)
                    #norm_advantage  = (_advantages - _advantages.mean()) / (_advantages.std() + 1e-10) 
                    prox1 = ratio * _advantages
                    prox2 = torch.clamp(ratio ,1-configs.epsilon,1+configs.epsilon) * _advantages
                    loss_policy = -torch.mean(torch.min(prox1,prox2)) #* 6e2
    
                    _entropy = -(policy_output * torch.log(policy_output + 1e-10)).sum(dim=2)
                    entropy = _entropy.sum(dim=1).mean() * 1e-3

                    total_loss = loss_policy + loss_critic - entropy 
    
                    self.network.optim.zero_grad()
                    total_loss.backward(retain_graph=True)
                    nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
                    xm.optimizer_step(self.network.optim)
                    xm.mark_step()
                    
            self.writter.add_scalar("Policy/entropy",entropy,traj)
            self.writter.add_scalar("Policy/loss policy",loss_policy,traj)

            self.writter.add_scalar("Value/values",_values.mean(),traj)
            self.writter.add_scalar("Value/vtarget",vtarget.mean(),traj)
            self.writter.add_scalar("Value/value loss",loss_critic,traj) 
            self.writter.add_scalar("Value/Explained variance",explained_variance,traj)

            self.writter.add_scalar("main/total loss",total_loss,traj)
            self.writter.add_scalar("main/epi rewards",torch.tensor([self.memory.traj_reward()[0]]).mean(),traj) 
            #self.writter.add_scalar("main/total steps",torch.tensor([self.memory.traj_reward()[1]]).mean(),traj)

            epreward = torch.tensor([self.memory.traj_reward()[0]]).mean().tolist()
            print(f"game {traj}/5000 | reward {round(epreward,4)} | entropy {entropy:.4f} | policy {loss_policy:.4f} | critic {loss_critic:.4f} | total {total_loss:.4f} ")
            if traj % 10 == 0 : # save every ...k steps
                self.save(traj)
                
        self.save("_end")
    
    @staticmethod
    def _train_fn():
        device = xm.xla_device()
        agnt = __class__(device)
        agnt.train(num_game=1400,batchsize=512,minibatch=16,optim_step=10) 

    @staticmethod
    def run_training(run=False):
        if run:
            xmp.spawn(agent._train_fn, nprocs=None, start_method="fork")

agent.run_training(True)